In [1]:
import numpy as np 
import pandas as pd

In [2]:
np.random.seed(123)

N = 1000
J = 4
T = 50

# Generate the data
np.random.seed(123)
mu = np.array([-1.71, 0.44, -1.37, -0.91, -1.23]).reshape(-1, 1)
sigma = np.diag(np.array([3.22, 3.24, 2.87, 4.15, 1.38])).reshape(5, 5)
print(mu, '\n')
print(sigma)


# generate the random parameters
betas = np.random.multivariate_normal(mu.flatten(), sigma, N)
print(betas.shape)

[[-1.71]
 [ 0.44]
 [-1.37]
 [-0.91]
 [-1.23]] 

[[3.22 0.   0.   0.   0.  ]
 [0.   3.24 0.   0.   0.  ]
 [0.   0.   2.87 0.   0.  ]
 [0.   0.   0.   4.15 0.  ]
 [0.   0.   0.   0.   1.38]]
(1000, 5)


In [3]:
price_transition_states = pd.read_csv(r'price_transition_states.csv')
price_transition_matrix = pd.read_csv(r'transition_prob_matrix.csv')

print(price_transition_states.head())
print(price_transition_matrix.head())

         X1        X2        X3        X4  X5  id
0  0.809830  2.730574  0.770491  2.966650   0   1
1  0.785441  2.453663  0.771422  2.964841   0   2
2  0.823926  2.238301  0.790901  2.914317   0   3
3  1.005571  1.581941  0.918171  2.084867   0   4
4  0.798525  2.693030  0.762935  2.690978   0   5
         V1        V2        V3        V4        V5        V6        V7  \
0  0.050283  0.020251  0.017505  0.003655  0.026568  0.006548  0.006352   
1  0.040564  0.023169  0.016224  0.003816  0.024936  0.007435  0.007776   
2  0.034444  0.016302  0.015208  0.004359  0.022437  0.006756  0.007631   
3  0.017591  0.008223  0.009868  0.007142  0.013897  0.006043  0.011003   
4  0.042309  0.016379  0.015619  0.003890  0.027125  0.006523  0.006811   

         V8        V9       V10  ...       V91       V92       V93       V94  \
0  0.008375  0.016022  0.013590  ...  0.017482  0.003807  0.008912  0.005710   
1  0.006751  0.014843  0.012902  ...  0.020574  0.003186  0.010063  0.005734   
2  0.0084

In [4]:
price_transition_states

,X1,X2,X3,X4,X5,id
0,0.809830,2.730574,0.770491,2.966650,0,1
1,0.785441,2.453663,0.771422,2.964841,0,2
2,0.823926,2.238301,0.790901,2.914317,0,3
3,1.005571,1.581941,0.918171,2.084867,0,4
4,0.798525,2.693030,0.762935,2.690978,0,5
...,...,...,...,...,...,...
95,1.096901,2.540894,0.933728,2.026588,0,96
96,1.228067,1.985250,0.941085,2.525047,0,97
97,1.118906,3.728712,1.055625,2.475710,0,98
98,0.847625,2.102348,1.363938,1.771260,0,99


In [5]:
price_transition_matrix

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100
0,0.050283,0.020251,0.017505,0.003655,0.026568,0.006548,0.006352,0.008375,0.016022,0.013590,...,0.017482,0.003807,0.008912,0.005710,0.016266,0.008533,0.006934,0.002275,0.001474,0.008172
1,0.040564,0.023169,0.016224,0.003816,0.024936,0.007435,0.007776,0.006751,0.014843,0.012902,...,0.020574,0.003186,0.010063,0.005734,0.013634,0.008189,0.006430,0.001822,0.001520,0.009950
2,0.034444,0.016302,0.015208,0.004359,0.022437,0.006756,0.007631,0.008431,0.019615,0.015308,...,0.016124,0.004657,0.010655,0.004589,0.014261,0.009842,0.009434,0.002486,0.001853,0.009703
3,0.017591,0.008223,0.009868,0.007142,0.013897,0.006043,0.011003,0.009424,0.022074,0.012562,...,0.010919,0.007949,0.013787,0.002806,0.011120,0.011316,0.016923,0.003837,0.003444,0.015517
4,0.042309,0.016379,0.015619,0.003890,0.027125,0.006523,0.006811,0.008304,0.017501,0.014749,...,0.016193,0.004160,0.009280,0.004994,0.017674,0.008824,0.008212,0.002280,0.001601,0.008316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.022406,0.010178,0.011546,0.005559,0.015755,0.005905,0.007434,0.010891,0.027838,0.015147,...,0.010508,0.008855,0.010696,0.003277,0.013045,0.012874,0.017172,0.004015,0.003121,0.010308
96,0.014125,0.006274,0.008583,0.005706,0.010867,0.005148,0.006759,0.012477,0.030100,0.013551,...,0.007031,0.013129,0.010076,0.002364,0.012457,0.014006,0.033324,0.005777,0.003969,0.009970
97,0.013230,0.006321,0.007653,0.004281,0.010171,0.004477,0.006573,0.022304,0.023152,0.009947,...,0.007212,0.014083,0.006911,0.002808,0.011422,0.010620,0.013875,0.008872,0.003258,0.008857
98,0.016398,0.007619,0.009392,0.006894,0.012828,0.006098,0.009410,0.009549,0.022639,0.012384,...,0.010010,0.010943,0.011931,0.002618,0.011045,0.012419,0.017758,0.004460,0.005623,0.013918


In [8]:
price_transition_matrix_np = price_transition_matrix.to_numpy()
price_transition_states_np = price_transition_states.to_numpy()

In [77]:
def simulate_markov(states, transition, T):
    state_indices = np.arange(states.shape[0])
    price_simu = np.zeros((T, 6))
    price_simu[0] = states[0]
    for t in range(1, T):
        preceding_state = price_simu[t-1, :]
        index_preceding_state = int(preceding_state[-1] - 1)
        price_simu[t, :] = np.random.choice(state_indices, p=(transition[index_preceding_state, :].flatten()))
    return price_simu


In [76]:
pd.DataFrame(simulate_markov(price_transition_states_np, price_transition_matrix_np, 50))

,0,1,2,3,4,5
0,0.80983,2.730574,0.770491,2.96665,0.0,1.0
1,21.00000,21.000000,21.000000,21.00000,21.0,21.0
2,41.00000,41.000000,41.000000,41.00000,41.0,41.0
3,91.00000,91.000000,91.000000,91.00000,91.0,91.0
4,30.00000,30.000000,30.000000,30.00000,30.0,30.0
5,93.00000,93.000000,93.000000,93.00000,93.0,93.0
6,60.00000,60.000000,60.000000,60.00000,60.0,60.0
7,57.00000,57.000000,57.000000,57.00000,57.0,57.0
8,8.00000,8.000000,8.000000,8.00000,8.0,8.0
9,35.00000,35.000000,35.000000,35.00000,35.0,35.0


In [63]:
def simulate_markov(states, transition, T):
    price_simu = np.zeros((T, len(states)))
    price_simu[0, :] = np.random.choice(states, p=transition[0, :])

    for t in range(1, T):
        print(f"Time step: {t}")
        index_preceding_state = int(price_simu[t-1, -1] - 1)
        print(f"Index of preceding state: {index_preceding_state}")
        print(f"Shape of states: {states.shape}")
        print(f"Shape of transition probabilities: {transition[index_preceding_state, :].shape}")
        price_simu[t, :] = np.random.choice(states, p=transition[index_preceding_state, :].flatten())

    return price_simu


In [61]:
simulate_markov(price_transition_states_np, price_transition_matrix_np, 50)

ValueError: a must be 1-dimensional